In [5]:
import json
import time
import pathlib
import numpy as np
import pandas as pd
from time import gmtime, strftime

import boto3
import sagemaker
from sagemaker.inputs import TrainingInput
from sagemaker.processing import ProcessingInput, ProcessingOutput, FeatureStoreOutput
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep

In [6]:
# Set region, boto3 and SageMaker SDK variables¶

#You can change this to a region of your choice
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity()["Account"]

random_state = 42

Using AWS Region: us-east-1


In [7]:
%store -r
%store

Stored variables and their in-db values:
clarify_data_uri                    -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
data_prefix                         -> 'sagemaker-tutorial/data'
default_bucket                      -> 'sagemaker-us-east-1-367158743199'
endpoint_config_name                -> 'sagemaker-tutorial-xgboost-smote-endpoint-config'
endpoint_name                       -> 'sagemaker-tutorial-xgboost-smote-endpoint'
feature_group_name                  -> 'FG-flow-sm-tutorial-31-16-16-17-9f41d66b'
header                              -> ['LABEL', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIA
hyperparameters                     -> {'max_depth': '5', 'eta': '0.2', 'gamma': '4', 'mi
local_processed_data                -> '../sm-tutorial/02_build_train/processed_data'
model_2_name                        -> 'sagemaker-tutorial-xgboost-smote'
model_data                          -> 's3://sagemaker-us-east-1-367158743199/tf2-resnet-
mp2_arn                             -> 'arn:a

# Step 1 - Data Wranlger Step